In [1]:
import nltk
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup
from future.utils import iteritems
import os

In [2]:
os.chdir(r"C:\Users\evkikum\OneDrive - Ericsson AB\UseCaseData\NLP\LayProgrammer")

In [3]:
wordnet_lemmatizer = WordNetLemmatizer()
stopwords = set(w.rstrip() for w in open('Data/stopwords.txt'))


In [4]:
positive_reviews = BeautifulSoup(open('electronics/positive.review').read(), features="html5lib")
positive_reviews = positive_reviews.findAll('review_text')

In [5]:
negative_reviews = BeautifulSoup(open('electronics/negative.review').read(), features='html5lib')
negative_reviews = negative_reviews.findAll('review_text')

In [6]:
word_index_map = {}
current_index = 0
positive_tokenized = []
negative_tokenized  = []
orig_reviews = []

In [7]:
def my_tokenizer(s):
    s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stopwords]
    return tokens

In [8]:
for review in positive_reviews:
    orig_reviews.append(review.text)
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [9]:
for review in negative_reviews:
    orig_reviews.append(review.text)
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [10]:
print("len(word_index_map)", len(word_index_map))

len(word_index_map) 10950


In [11]:
## lets create our input dataframe
def token_to_vectors(tokens, label):
    x = np.zeros(len(word_index_map) + 1)
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x/x.sum()
    x[-1] = label
    return x

In [12]:
df = pd.DataFrame()
df_positive = pd.DataFrame()
df_negative = pd.DataFrame()


In [13]:
for tokens in positive_tokenized:
    data = pd.DataFrame()
    data['x'] = token_to_vectors(tokens, 1)
    data = data.T
    data = data.reset_index()
    data = data.drop(['index'], axis = 1)
    df_positive = df_positive.append(data)
    
    

In [14]:
df_positive = df_positive.reset_index()
df_positive = df_positive.drop(['index'], axis = 1)

In [ ]:
for tokens in negative_tokenized:
    data = pd.DataFrame()
    data['x'] = token_to_vectors(tokens, 0)
    data = data.T
    data = data.reset_index()
    data = data.drop(['index'], axis = 1)
    df_negative = df_negative.append(data)

In [ ]:
df_negative = df_negative.reset_index()
df_negative = df_negative.drop(['index'], axis = 1)

In [ ]:
df = df.append(df_positive)
df = df.append(df_negative)
df = df.reset_index()
df = df.drop(['index'], axis = 1)

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:,-1]

In [ ]:
Xtrain = X.iloc[:-100,:]
Xtest = X.iloc[-100:,:]
ytrain = y.iloc[:-100]
ytest = y.iloc[-100:]

In [ ]:
model = LogisticRegression()
model.fit(Xtrain, ytrain)
model.score(Xtrain, ytrain)

In [ ]:
model.score(Xtest, ytest)